<a href="https://colab.research.google.com/github/loepin/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Needless to say, you can run all the way!

This script was created by **[Tony is Tucker](https://wxp123.me)**, only for learning, don't abuse it. If in doubt, pull to the end!

In [0]:
#@title Check GPU
# View device, is K80 or T4, if it is K80...！
! /opt/bin/nvidia-smi

# First step, ready workspace
You can choose two ways in this step.
1. With the default workspace, you don't need to upload it yourself, just for familiarity with the operation.
2. Upload your workspace to the specified directory via Google Drive.



Google Cloud Disk Address: https://drive.google.com/drive/my-drive
Note: Google's website now requires "scientific internet access" to access.  

In [0]:
#@title Start mounting
#Mount Google Cloud Drive
#Click the link authorization, copy the authorization code, fill in the box, and then press Enter.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Create a DeepFaceLab directory and go to the directory
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


After the mount is successful, you can load an official workspace or upload your own workspace via Google Cloud.
Note that the uploaded zip file is in zip format. The file name must be workspace.zip
It is recommended that this workspace only contains src, dst, etc., does not contain the model, so the sixth will not overwrite the model file.
The model file is relatively small and can be directly uploaded to the model directory without compression.

In [0]:
#@title Load the sample project workspace
# 1. Hands-on can use this line, directly git clone a workspace
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

In [0]:
#@title Unzip your project workspace
#You need to upload the workspace.zip to the DeepFaceLab directory of Google Cloud.
!unzip  workspace.zip

#Step 2 Install DeepFaceLab
Get the source code, install the dependencies, choose the version according to your situation
620: stable version, a better version
1019: Support for SAEHD version
Last: latest version 

In [0]:
#@title Start Installation

Version = "620" #@param ["620", "1019", "last"]
!rm -fr DeepFaceLab_Colab
if Version=="620":
  print("620 Version loading....")
  # Get the DFL source code v1.6.1 stable version
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git  
elif Version=="1019":
  print("1019 Version loading....")
  !git clone -b v1.10.19 https://github.com/dream80/DeepFaceLab_Colab.git  
else:
  print("The latest version is loading....")
  !git clone https://github.com/dream80/DeepFaceLab_Colab.git

# Go to the DeepFaceLab_Colab directory
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# Install Python dependencies
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# Step 3. Extract the face

In [0]:
#@title Start extraction
target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  


In [0]:
#@title Start sorting
target = "src" #@param ["src","dst"]
order = "hist" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #Src Sort, you can view the results through Google Cloud Drive, delete bad images
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

# Step 4. Training model


* Support models such as H128, SAE, DF, LIAEF128, and select models according to their own situation.
* The first time you need to configure the model parameters, do not understand the direct return default.
* If you don't want to train, you can click to stop. When you stop, an exception will be thrown, but it doesn't matter. Can continue training next time



In [0]:
#@title Start training
Model = "H128" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

**Model preview method please refer to:**
Https://www.wxp123.me/p/329

Brief description: left->file->/driver/..../model/ *preview*.jpg->double click
  
This method is more convenient than previewing in a script and previewing on a web drive, and there is no need to launch a preview configuration!

# Step 5. Convert the output

What is the candidate for training the model, what to choose here
For example, training H128, here is H128


In [0]:
#@title Start converting images

Model = "H128" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [0]:
#@title Start picture to video
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#Step 6. Continue training
You don't need to perform all the above steps when you start training for the second time, or continue training after dropping. Just a few simple steps are as follows.
Mount the cloud disk
2. Installation dependencies
3. Start training
**Note: **workspace.zip only contains src and dst, no model, otherwise the model will be overwritten. ! ! !

In [0]:
#@title One-button operation
#Mount Google Cloud Drive
#Click the link authorization, copy the authorization code, fill in the box, and then press Enter.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Re-covering decompression and increasing loading speed saves a lot of time
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  -o workspace.zip


# Go to the DeepFaceLab_Colab directory
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# Install Python dependencies
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image

# Start training SAE. If it is another model, modify the following parameters, such as H128.

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE --no-preview

# Tool


When using the script to practice the model, you will often need to upload, unzip, compress, download, etc. The following provides a decompressed and compressed script for everyone.
All compressed files exist in the zip directory, the name is called data.zip, select the contents under the folder before compression.
When extracting, upload a data.zip and choose where to extract it.



In [0]:
#@title Adding...

# other

 Google Cloud Address: https://drive.google.com/drive/my-drive

 Author email :wpgdream@gmail.com
 
 Use the tutorial: https://www.wxp123.me
 
 
Don't ask for anything else, just ask for a little star in github ^_^

https://github.com/dream80/DeepFaceLab_Colab

Top right corner thank you!!!